In [1]:
import torch as th
import syft as sy 
hook = sy.TorchHook(th)

alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

x = th.tensor([0.1, 0.2, 0.3])

x = x.fix_prec()
x = x.share(alice, bob, secure_worker)

In [2]:
import random

Q = 121639451781281043402593

def encrypt(x, n_shares = 2):
    shares = list()
    
    for i in range(n_shares-1):
        shares.append(random.randint(0,Q))
        
    final_share = Q - (sum(shares) % Q) + x
    
    share.append(final_share)
    
    return tuple(shares)
    
def decrypt(shares):
    return sum(shares) % Q
    
def add(a, b):
    c = list()
    
    assert(len(a) == len(b))
    
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
        
    return tuple(c)

In [3]:
def generate_mul_triple():
    a = random.randrange(Q)
    b = random.randrange(Q)
    a_mul_b = (a * b) % Q
    return encrypt(a), encrypt(b), encrypt(a_mul_b)
    
#we also assume that the crypto provider distributes the shares

In [4]:
def mul(x, y):
    a, b, a_mul_b = generate_mul_triple()
    
    #x remains hidden because a is random 
    alpha = decrypt(x - a)
    #<-y remains hidden because b is random 
    beta  = decrypt(y - b)
    
    #local re-combination
    return alpha.mul(beta) + alpha.mul(b) + a.mul(beta) + a_mul_b

In [5]:
BASE=10
PRECISION=4

def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION
    
encode(3.5) 
#<-- 35000
decode(35000) 
#<-- 3.5

3.5

In [6]:
#WHAT IS SECURE MULTI-PARTY COMPUTATION?
#https://blog.openmined.org/what-is-secure-multi-party-computation/